# Detect Emergency Vehicles - Transfer learning

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import os
import random
from shutil import copyfile

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

### Getting the data

Data has been provided by the organizers of the hackathon :
https://datahack.analyticsvidhya.com/contest/janatahack-computer-vision-hackathon/


In [6]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!ls "/content/drive/My Drive/BigData2020/data"
dirPath = "/content/drive/My Drive/Colab Notebooks/AV/AV_Janata_Hack_3/data/train/"
#%ls -l "$dirPath" # Get the content of the directory in long format (note the '$')

In [0]:
!rm -r data

In [0]:
#Source - https://gist.github.com/bertcarremans/679624f369ed9270472e37f8333244f5
def img_train_test_split(img_source_dir, train_size):
    """
    Randomly splits images over a train and validation folder, while preserving the folder structure
    
    Parameters
    ----------
    img_source_dir : string
        Path to the folder with the images to be split. Can be absolute or relative path   
        
    train_size : float
        Proportion of the original images that need to be copied in the subdirectory in the train folder
    """    
    if not (isinstance(img_source_dir, str)):
        raise AttributeError('img_source_dir must be a string')
        
    if not os.path.exists(img_source_dir):
        raise OSError('img_source_dir does not exist')
        
    if not (isinstance(train_size, float)):
        raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
    if not os.path.exists('data'):
        os.makedirs('data')
    if not os.path.exists('data/train'):
        os.makedirs('data/train')
    if not os.path.exists('data/validation'):
        os.makedirs('data/validation')
            
    # Get the subdirectories in the main image folder
    subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]

    for subdir in subdirs:
        subdir_fullpath = os.path.join(img_source_dir, subdir)
        if len(os.listdir(subdir_fullpath)) == 0:
            print(subdir_fullpath + ' is empty')
            break

        train_subdir = os.path.join('data/train', subdir)
        validation_subdir = os.path.join('data/validation', subdir)

        # Create subdirectories in train and validation folders
        if not os.path.exists(train_subdir):
            os.makedirs(train_subdir)

        if not os.path.exists(validation_subdir):
            os.makedirs(validation_subdir)

        train_counter = 0
        validation_counter = 0

        # Randomly assign an image to train or validation folder
        for filename in os.listdir(subdir_fullpath):
            if filename.endswith(".jpg") or filename.endswith(".png"): 
                fileparts = filename.split('.')

                if random.uniform(0, 1) <= train_size:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(train_subdir, str(train_counter) + '.' + fileparts[1]))
                    train_counter += 1
                else:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(validation_subdir, str(validation_counter) + '.' + fileparts[1]))
                    validation_counter += 1
                    
        print('Copied ' + str(train_counter) + ' images to data/train/' + subdir)
        print('Copied ' + str(validation_counter) + ' images to data/validation/' + subdir)

In [0]:
img_train_test_split(dirPath,0.80)

In [0]:
train=pd.read_csv(dirPath+'train.csv', index_col='image_names')
test=pd.read_csv(dirPath+'test.csv', index_col='image_names')

In [0]:
labels = list(train['emergency_or_not'].unique())

for label in labels:
  try:
    os.makedirs('./drive/My Drive/Colab Notebooks/AV/AV_Janata_Hack_3/data/train/'+str(label))
  except:
    print('Directory already exists/ Error in creation')

In [0]:
for image in train.index:
  folder = train.loc[image, 'emergency_or_not']
  src = './drive/My Drive/Colab Notebooks/AV/AV_Janata_Hack_3/'+'images/{}'.format(image)
  dst = './drive/My Drive/Colab Notebooks/AV/AV_Janata_Hack_3/data/train/{}/{}'.format(folder, image)
  try:
    copyfile(src, dst)
  except:
    print('Could not copy {} - {}'.format(image,folder))

In [0]:
for image in test.index:
  src = './drive/My Drive/Colab Notebooks/AV/AV_Janata_Hack_3/'+'images/{}'.format(image)
  dst = './drive/My Drive/Colab Notebooks/AV/AV_Janata_Hack_3/data/test/{}'.format(image)
  try:
    copyfile(src, dst)
  except:
    print('Could not copy {} - {}'.format(image,folder))

### Standardizing the data and Augmentation

Our raw images have a variety of sizes. In addition, each pixel consists of 3 integer
values between 0 and 255 (RGB level values). This isn't a great fit for feeding a
 neural network. We need to do 2 things:

- Standardize to a fixed image size. We pick 150x150.
- Normalize pixel values between -1 and 1. We'll do this using a `Normalization` layer as
 part of the model itself.

Here, we'll do image resizing in the data pipeline (because a deep neural network can
only process contiguous batches of data), and we'll do the input value scaling as part
 of the model, when we create it.

Let's resize images to 150x150:


In [111]:
train_dir='./data/train/'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1333 images belonging to 2 classes.


In [3]:
validation_dir='./data/validation/'

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

NameError: ignored

### Building the model using pre-trained VGG16 model on Imagenet

In [118]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()
# make a reference to VGG's input layer
inp = vgg.input
num_classes=2

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it

out = layers.Dropout(0.3)(vgg.layers[-2].output)
out = new_classification_layer(out)
# create a new network between inp and out
model_new = Model(inp, out)

# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [123]:
epochs = 10
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=epochs, 
                    verbose=1,
                    steps_per_epoch=1333/20, # batch size of 20 is defined in generator, so 2000/20 training batches
                    validation_steps=313/20,callbacks=[callback])

Epoch 1/10
67/66 [==============================] - 38s 574ms/step - loss: 0.3423 - acc: 0.8792 - val_loss: 0.5008 - val_acc: 0.8498
Epoch 2/10
67/66 [==============================] - 39s 581ms/step - loss: 0.3386 - acc: 0.8740 - val_loss: 0.3563 - val_acc: 0.8466
Epoch 3/10
67/66 [==============================] - 39s 581ms/step - loss: 0.3394 - acc: 0.8552 - val_loss: 0.6951 - val_acc: 0.8275
Epoch 4/10
67/66 [==============================] - 40s 590ms/step - loss: 0.3014 - acc: 0.8905 - val_loss: 0.4296 - val_acc: 0.8498


In [2]:
test_dir='./drive/My Drive/Colab Notebooks/AV/AV_Janata_Hack_3/data/.'
test_datagen = ImageDataGenerator(
    rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=1,class_mode='categorical',classes=['test']
        # Since we use binary_crossentropy loss, we need binary labels),
)

NameError: ignored

In [1]:
filenames = test_generator.filenames
nb_samples = len(filenames)
predictions = model.predict_generator(test_generator,steps = nb_samples)

NameError: ignored

In [0]:
score = predictions*100

In [84]:
score

array([[1.35688076e-03],
       [5.39132195e+01],
       [3.64004745e+01],
       [2.22637672e+01],
       [1.00000000e+02],
       [4.56496984e-01],
       [9.64460373e+00],
       [9.87462234e+01],
       [8.18789291e+01],
       [7.48443699e+00],
       [9.99787140e+01],
       [1.00000000e+02],
       [2.84591377e-01],
       [6.93845987e+00],
       [6.13610611e+01],
       [1.35688076e-03],
       [1.21367965e-02],
       [2.98833256e+01],
       [1.06690254e+01],
       [3.01123667e+00],
       [1.00000000e+02],
       [7.27723837e+00],
       [1.64489344e-01],
       [1.22174573e+00],
       [9.99999771e+01],
       [2.54815251e-01],
       [9.92493591e+01],
       [1.79843903e-02],
       [7.79587173e+01],
       [2.27890229e+00],
       [1.64850593e+00],
       [1.18470514e+00],
       [9.80174408e+01],
       [3.06888849e-01],
       [1.41668642e+00],
       [9.38022017e-01],
       [2.13955593e+00],
       [2.24709682e+01],
       [9.54863815e+01],
       [7.00901413e+01],


In [0]:
predictions_df=pd.DataFrame(data=filenames, columns=["filename"])

In [0]:
predictions_df['score']=score

In [0]:
predictions_df.to_csv('submission.csv')

In [74]:
pd['filename']=filenames
pd['scores']=score

TypeError: ignored